In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('../data/final_data_corrected.csv')
data.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [3]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTHG']

In [4]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [5]:
#data[data.isna().any(axis=1)]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [7]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('lr', LinearRegression())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}')

R2 Score: -1.7819033041015238
MAE: 1.7827097039473685


In [8]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(sparse_output = False), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}')

R2 Score: -0.03616380636140293
MAE: 1.1260830716780856


In [9]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data['FTAG']

In [10]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [12]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}') 

R2 Score: -0.10741131087748834
MAE: 0.9463610690453956


In [24]:
pd.set_option('display.max_columns', None)
data1 = pd.read_csv('../data/data_betting.csv')
data1.head(1)

,Unnamed: 0,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,0,0,4.2,3.6,1.85,4.33,3.5,1.87,4.3,3.55,1.85,4.5,3.65,1.89,4.4,3.5,1.83,4.6,3.5,1.87,4.6,3.78,1.95,4.39,3.59,1.88,2.1,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.5,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.5,3.6,1.8,4.5,3.5,1.83,4.4,3.55,1.85,4.58,3.63,1.88,4.8,3.4,1.78,4.75,3.5,1.85,5.01,3.7,1.91,4.56,3.57,1.85,2.1,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.5,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85


In [25]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data['FTHG']

In [26]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [28]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}') 

R2 Score: -0.045067825637107095
MAE: 1.1356998194984596


In [30]:
X = data[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
          'PSH', 'PSD', 'PSA', 'WHH','WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH', 'AvgD', 'AvgA', 'B365>2.5',
          'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH','B365AHA', 'PAHH', 'PAHA', 
          'MaxAHH', 'MaxAHA', "AvgAHH", 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA',
          'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD',
          'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
          'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']].copy()
y = data['FTAG']

In [31]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [33]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}')

R2 Score: -0.047987643849763284
MAE: 0.948820378594154


In [34]:
pd.set_option('display.max_columns', None)
data2 = pd.read_csv('../data/data_teams.csv')
data2.head(1)

,Date,Time,TeamH,TeamA,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HStr,HSOH,HSOA,HSAH,HSAA,HSDH,HSDA,HO,HA,HM,HD,HDP,HIP,HSAAg,HATAAg,HGls,HAssts,HGlsPK,HxG,HNpxG,HxAG,HPoss,HPrg,HPrgP,HGKS%,HGKCS%,HSoT,Hg-xG,HCmp,HCmp%,HPrgDist,HXa,HKP,HPPA,HCrsPA,HTB,HCrs,HCK,HSCA,HGCA,HTkIW,HLost,HBlocks,HPass,HInt,HClr,HAttPen,HSucc%,HCPA,HDis,HPrgR,HOnG,HOnGA,HOnxG,HOnxGA,AStr,ASOH,ASOA,ASAH,ASAA,ASDH,ASDA,AO,AA,AM,AD,ADP,AIP,ASAAg,ATAAg,AGls,AAssts,AGlsPK,AxG,ANpxG,AxAG,APoss,APrg,APrgP,AGKS%,AGKCS%,ASoT,Ag-xG,ACmp,ACmp%,APrgDist,AXa,AKP,APPA,ACrsPA,ATB,ACrs,ACK,ASCA,AGCA,ATkIW,ALost,ABlocks,APass,AInt,AClr,AAttPen,ASucc%,ACPA,ADis,APrgR,AOnG,AOnGA,AOnxG,AOnxGA
0,5/8/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,3,1085,1100,1110,1110,1060,1090,76,77,75,76,4,2,26.6,25.2,1.0,0.76,0.97,1.02,0.96,0.75,45.8,14.2,32.2,71.8,24.3,3.51,-0.02,348.8,77.2,2310.4,0.61,8.19,6.14,1.27,0.95,15.4,4.89,19.4,1.76,11.2,6.73,12.3,9.08,9.65,21.5,19.9,46.9,5.41,12.1,32.1,1.05,1.3,1.02,1.29,4,1245,1285,1250,1250,1240,1320,80,83,81,79,7,7,24.1,23.0,2.14,1.62,2.05,1.87,1.79,1.39,59.5,21.7,54.2,70.6,35.1,5.03,0.27,482.8,83.1,2658.9,1.21,11.7,12.01,1.65,2.24,17.8,5.81,27.6,3.81,8.89,6.51,9.49,7.22,6.35,15.8,34.0,46.3,7.38,10.1,53.5,2.24,1.16,1.87,1.12


In [35]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTHG']

In [36]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [38]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}') 

R2 Score: 0.06004131626903175
MAE: 1.0761761890456925


In [39]:
X = data2[['Date', 'Time', 'TeamH', 'TeamA', 'FTR', 'HTR', 'Referee', 'HStr', 'HSOH', 'HSOA', 'HSAH', 'HSAA', 'HSDH',
          'HSDA', 'HO', 'HA', 'HM', 'HD', 'HDP', 'HIP', 'HSAAg', 'HATAAg', 'HGls', 'HAssts', 'HGlsPK', 'HxG', 'HNpxG', 'HxAG',
          'HPoss', 'HPrg', 'HPrgP', 'HGKS%', 'HGKCS%', 'HSoT', 'Hg-xG', 'HCmp', 'HCmp%', 'HPrgDist', 'HXa', 'HKP', 'HPPA', 'HCrsPA',
          'HTB', 'HCrs', 'HCK', 'HSCA', 'HGCA', 'HTkIW', 'HLost', 'HBlocks', 'HPass', 'HInt', 'HClr', 'HAttPen', 'HSucc%', 'HCPA',
          'HDis', 'HPrgR', 'HOnG', 'HOnGA', 'HOnxG', 'HOnxGA', 'AStr', 'ASOH', 'ASOA', 'ASAH', 'ASAA', 'ASDH', 'ASDA', 'AO', 'AA',
          'AM', 'AD', 'ADP', 'AIP', 'ASAAg', 'ATAAg', 'AGls', 'AAssts', 'AGlsPK', 'AxG', 'ANpxG', 'AxAG', 'APoss', 'APrg', 'APrgP',
          'AGKS%', 'AGKCS%', 'ASoT', 'Ag-xG', 'ACmp', 'ACmp%', 'APrgDist', 'AXa', 'AKP', 'APPA', 'ACrsPA', 'ATB', 'ACrs', 'ACK',
          'ASCA', 'AGCA', 'ATkIW', 'ALost', 'ABlocks', 'APass', 'AInt', 'AClr','AAttPen', 'ASucc%', 'ACPA', 'ADis', 'APrgR', 'AOnG',
          'AOnGA', 'AOnxG', 'AOnxGA']].copy()
y = data2['FTAG']

In [40]:
X = X.drop(columns = ['Date', 'Time', 'FTR', 'HTR'])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

In [42]:
linreg = Pipeline(
    steps = [
        ('ct', ColumnTransformer(
            transformers = [
                ('ohe', OneHotEncoder(), ['TeamH', 'TeamA', 'Referee'])
            ],
            remainder = 'passthrough'
        )
        ),
        ('scaler', MinMaxScaler()),
        ('linreg',  RidgeCV())
    ]
)

linreg.fit(X_train, y_train)

print(f'R2 Score: {r2_score(y_test, linreg.predict(X_test))}')
print(f'MAE: {mean_absolute_error(y_test, linreg.predict(X_test))}')

R2 Score: -0.11447561385868732
MAE: 0.9514405211714799
